In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation 
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
import tensorflow_hub as hub

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



## Import Dataset

In [2]:
dataset_name = "cifar10"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

IMG_SIZE = 224
batch_size = 64
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

## Import Model

In [3]:
hub_url = 'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0/feature-vector'
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[IMG_SIZE, IMG_SIZE, 3]),
    hub.KerasLayer(hub_url, trainable=True),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation='softmax'),
])
model.build((None, IMG_SIZE, IMG_SIZE, 3))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              5919312   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 5,932,122
Trainable params: 5,871,514
Non-trainable params: 60,608
_________________________________________________________________


## Transfer Learning on Model

In [4]:
model_name = 'enetv2b0.h5'
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(ds_train,
              epochs=5,
              validation_data=ds_test,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/5
781/781 [==============================] - 81s 93ms/step - loss: 0.9298 - accuracy: 0.8334 - val_loss: 0.6638 - val_accuracy: 0.9413
Epoch 2/5
781/781 [==============================] - 72s 92ms/step - loss: 0.6679 - accuracy: 0.9422 - val_loss: 0.6144 - val_accuracy: 0.9590
Epoch 3/5
781/781 [==============================] - 72s 92ms/step - loss: 0.6144 - accuracy: 0.9659 - val_loss: 0.6001 - val_accuracy: 0.9623
Epoch 4/5
781/781 [==============================] - 72s 92ms/step - loss: 0.5875 - accuracy: 0.9764 - val_loss: 0.5936 - val_accuracy: 0.9651
Epoch 5/5
781/781 [==============================] - 72s 92ms/step - loss: 0.5689 - accuracy: 0.9842 - val_loss: 0.5886 - val_accuracy: 0.9665


In [8]:
tf.saved_model.save(model,'enetv2b0')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: enetv2b0/assets


INFO:tensorflow:Assets written to: enetv2b0/assets


In [9]:
new_model = tf.keras.models.load_model('enetv2b0/')

In [11]:
new_model.evaluate(ds_test)

156/156 [==============================] - 5s 26ms/step - loss: 0.5886 - accuracy: 0.9665


[0.5885660648345947, 0.9665464758872986]